
# Standalone Transformers SFT Notebook (Chat-format, Assistant-only Loss)

This notebook trains a causal language model using **Hugging Face Transformers** (no TRL).  
It expects a dataset with a `messages` column: a list of chat turns like
```json
{"messages": [
  {"role": "system", "content": "You are helpful."},
  {"role": "user", "content": "Say hi."},
  {"role": "assistant", "content": "Hello!"}
]}
```
**Features**
- Uses the model's `chat_template` (`tokenizer.apply_chat_template`) to format messages.
- Masks the loss to **only** the last assistant turn (assistant-only loss) without TRL.
- Optional **LoRA** (PEFT) and optional **4-bit** quantization (bitsandbytes).
- Works with Qwen/TinyLlama/phi-3-mini etc. (set `MODEL_ID`).
- Includes a **toy dataset** if you don't have one yet.

> Tip: If your `content` pieces are structured (e.g., list of parts), the normalizer flattens text parts.


In [1]:
%pip install nbstripout
%pip install -U "huggingface-hub>=0.34.0,<1.0"
%pip check
%pip install hf_transfer
%pip install -U  datasets accelerate peft trl bitsandbytes peft flash-attn
%pip install -U "trl>=0.10.0" "transformers>=4.44.0"
from datasets import load_dataset
from transformers import AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
!export HF_HUB_ENABLE_HF_TRANSFER=1
import gc
from google.colab import runtime
from google.colab import auth
auth.authenticate_user()
import math
import json, random
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer, pipeline, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model, TaskType
import os
import wandb
import shutil
import torch
from datetime import datetime
from google.colab import userdata
from huggingface_hub import login, whoami
os.environ["WANDB_DISABLED"] = "false"  # or "true" to mute
os.environ["WANDB_PROJECT"]   = "qwen3coder-finetune-fp16"

os.environ["WANDB_LOG_MODEL"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
REPO_URL="https://github.com/UH-Insure/Colab-Training.git"
REPO="Colab-Training"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    %pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    %pip install -e .


ipython 7.34.0 requires jedi, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 108.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 54.9 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:


In [ ]:
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
WANDB_TOKEN = userdata.get('WANDB_KEY')
os.environ["WANDB_API_KEY"] = WANDB_TOKEN
os.environ["HF_TOKEN"] = HF_TOKEN
wandb.login(key=WANDB_TOKEN, relogin=True)
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: joshuareedenterprises (joshuareedenterprises-university-of-houston) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Logged in as: j05hr3d


In [ ]:

import os, json, random
from dataclasses import dataclass
from typing import List, Dict, Any

# Disable Torch Dynamo / torch.compile to avoid environment-specific issues.
os.environ.setdefault("TORCH_COMPILE_DISABLE", "1")

import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments, BitsAndBytesConfig
)

print("Python:", os.sys.version)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Torch: 2.8.0+cu126
CUDA available: True
Device: NVIDIA A100-SXM4-80GB


In [ ]:

# ==== USER CONFIG ====
MODEL_ID = "Qwen/Qwen3-Coder-30B-A3B-Instruct"
ADAPTER_ID = "tam2003/Qwen3-Coder-30b-v5-2ep"
TRUST_REMOTE_CODE = True  # set True for models (e.g., Qwen) that need remote code
OUTPUT_DIR = "tam2003/SFT-Qwen3-30B-dsetV7"

# Data: provide a path to a JSONL/JSON with a `messages` field per record
JSONL_PATH = "data/nocomments_message_format_v7.jsonl"   # e.g., "/content/my_data.jsonl" (leave blank to use toy dataset)

# Training hyperparameters
MAX_SEQ_LEN = 4096            # adjust to your context length + memory
BATCH_SIZE = 2                # per-device train batch size
GRAD_ACC = 4
EPOCHS = 10
LR = 2e-4
EVAL_STEPS = 30
SAVE_STEPS = 30
LOG_STEPS = 30

# LoRA options
USE_LORA = True
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
# Narrow target modules list if memory-limited. Common: q_proj, v_proj, k_proj, o_proj
#LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]
LORA_TARGET_MODULES = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
#LORA_TARGET_MODULES = ["q_proj","k_proj","v_proj"]  # lora_target_modules

# bitsandbytes config
USE_NESTED_QUANT = True  # use_nested_quant
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"  # bnb_4bit_compute_dtype

# 4-bit options
USE_4BIT = True
BNB_COMPUTE_DTYPE = torch.bfloat16 if torch.cuda.is_available() else torch.float32


In [ ]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, trust_remote_code=TRUST_REMOTE_CODE)
# Ensure pad token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Pad token id:", tokenizer.pad_token_id, "EOS:", tokenizer.eos_token_id)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Pad token id: 151643 EOS: 151645


In [ ]:
from src.data import load_or_make_dataset, _chunk_messages_by_tokens, _normalize_messages, explode_long_conversations

raw = load_or_make_dataset(JSONL_PATH)
raw = raw.filter(lambda ex: ex["set"] in ["unsupervised", "supervised"])
print(raw[:1])

{'filetype': ['cryptol'], 'messages': [[{'role': 'system', 'content': 'Return exactly ONE fenced code block labeled `cryptol` and nothing else (no prose before/after).\n'}, {'role': 'user', 'content': "Write Cryptol properties to verify the behavior of the 'splitAt' function when applied to the sequence 'x'. Ensure the properties check that the output tuple correctly splits 'x' into two subsequences of equal length, and that concatenating these subsequences reconstructs the original sequence 'x'."}, {'role': 'assistant', 'content': '```cryptol\nx = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([2][8],[2][8])\n\na = y.0 @ 0\nb = y.0 @ 1\nc = y.1 @ 0\nd = y.1 @ 1\n```'}]]}


In [ ]:
# --- actually apply the splitting to `raw` using MAX_SEQ_LEN = 4096 ---
raw = explode_long_conversations(raw, tokenizer, MAX_SEQ_LEN)
print("After splitting:")
print(raw[:1])

Skipping over-long single message with 10284 tokens
Skipping over-long single message with 77173 tokens
Skipping over-long single message with 5842 tokens
Skipping over-long single message with 15089 tokens
Skipping over-long single message with 7648 tokens
Skipping over-long single message with 20401 tokens
Skipping over-long single message with 8464 tokens
Skipping over-long single message with 10224 tokens
Skipping over-long single message with 6912 tokens
Skipping over-long single message with 5095 tokens
Skipping over-long single message with 8968 tokens
Skipping over-long single message with 5448 tokens
Skipping over-long single message with 4680 tokens
Skipping over-long single message with 4233 tokens
Skipping over-long single message with 13673 tokens
Skipping over-long single message with 13655 tokens
Skipping over-long single message with 6338 tokens
Skipping over-long single message with 5266 tokens
Skipping over-long single message with 10475 tokens
Skipping over-long sing

In [ ]:
def encode_chat_last_assistant_only(messages: List[Dict[str, Any]], max_len: int):
    msgs = _normalize_messages(messages)

    # find the last assistant turn
    last_asst_idx = -1
    for i in range(len(msgs) - 1, -1, -1):
        if msgs[i].get("role") == "assistant":
            last_asst_idx = i
            break

    if last_asst_idx == -1:
        # no assistant -> skip this example
        return None

    # history without the last assistant, and history with it
    hist_wo = msgs[:last_asst_idx]          # may be []
    hist_w  = msgs[:last_asst_idx + 1]      # at least one message (the assistant)

    # handle empty history safely
    if hist_wo:
        ids_hist = tokenizer.apply_chat_template(
            hist_wo,
            tokenize=True,
            add_generation_prompt=False,
        )
    else:
        ids_hist = []

    # full sequence including last assistant
    ids_full = tokenizer.apply_chat_template(
        hist_w,
        tokenize=True,
        add_generation_prompt=False,
    )

    # Ensure EOS at the end (optional but nice)
    if tokenizer.eos_token_id is not None and (len(ids_full) == 0 or ids_full[-1] != tokenizer.eos_token_id):
        ids_full = ids_full + [tokenizer.eos_token_id]

    input_ids = ids_full

    # labels: mask out the history, keep only the last assistant tokens
    labels = [-100] * len(ids_hist) + input_ids[len(ids_hist):]
    attention_mask = [1] * len(input_ids)

    # Truncate from the left if needed
    if len(input_ids) > max_len:
        input_ids = input_ids[-max_len:]
        attention_mask = attention_mask[-max_len:]
        labels = labels[-max_len:]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }


def ds_map_fn(ex):
    out = encode_chat_last_assistant_only(ex["messages"], MAX_SEQ_LEN)
    if out is None:
        # IMPORTANT: always return all keys, even for rows we will drop
        return {
            "input_ids": [],
            "attention_mask": [],
            "labels": [],
            "_drop": True,
        }
    out["_drop"] = False
    return out


raw = raw.class_encode_column("filetype")

split = raw.train_test_split(test_size=0.2, seed=42, stratify_by_column="filetype")
train_raw, eval_raw = split["train"], split["test"]

train_ds = train_raw.map(
    ds_map_fn,
    remove_columns=[c for c in train_raw.column_names if c != "messages"],
)
train_ds = train_ds.filter(lambda ex: ex["_drop"] is False).remove_columns("_drop")

eval_ds = eval_raw.map(
    ds_map_fn,
    remove_columns=[c for c in eval_raw.column_names if c != "messages"],
)
eval_ds = eval_ds.filter(lambda ex: ex["_drop"] is False).remove_columns("_drop")

# quick inspection
print("Train example lens:", len(train_ds[0]["input_ids"]), len(train_ds[0]["labels"]))
print("Eval example lens:", len(eval_ds[0]["input_ids"]), len(eval_ds[0]["labels"]))

max_train_len = max(len(ex["input_ids"]) for ex in train_ds)
max_eval_len  = max(len(ex["input_ids"]) for ex in eval_ds)
print("Max train len:", max_train_len)
print("Max eval  len:", max_eval_len)

print(train_ds, eval_ds)

Casting to class labels:   0%|          | 0/896 [00:00<?, ? examples/s]

Map:   0%|          | 0/716 [00:00<?, ? examples/s]

Filter:   0%|          | 0/716 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Filter:   0%|          | 0/180 [00:00<?, ? examples/s]

Train example lens: 139 139
Eval example lens: 239 239
Max train len: 4034
Max eval  len: 4042
Dataset({
    features: ['messages', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 667
}) Dataset({
    features: ['messages', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 169
})


In [ ]:

@dataclass
class DataCollatorForCausalLMWithLabels:
    tokenizer: AutoTokenizer
    label_pad_token_id: int = -100

    def __call__(self, features: List[Dict[str, Any]]):
        # Separate out labels for manual padding
        labels = [f["labels"] for f in features]
        batch_inputs = {
            "input_ids": [f["input_ids"] for f in features],
            "attention_mask": [f["attention_mask"] for f in features]
        }
        padded = self.tokenizer.pad(
            batch_inputs, padding=True, return_tensors="pt"
        )
        max_len = padded["input_ids"].shape[1]
        padded_labels = torch.full(
            (len(labels), max_len), self.label_pad_token_id, dtype=torch.long
        )
        for i, lab in enumerate(labels):
            padded_labels[i, :len(lab)] = torch.tensor(lab, dtype=torch.long)
        padded["labels"] = padded_labels
        return padded

data_collator = DataCollatorForCausalLMWithLabels(tokenizer=tokenizer)


In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect(); torch.cuda.empty_cache()

In [ ]:

# 4-bit quantization
compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=USE_NESTED_QUANT,
)

base = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        load_in_8bit=False,
        quantization_config=bnb_config,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=False,
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
)

base = prepare_model_for_kbit_training(base)

# Ensure model has pad token id in config for generation convenience
if getattr(base.config, "pad_token_id", None) is None:
    base.config.pad_token_id = tokenizer.pad_token_id


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [ ]:
blk = base.model.layers[0]           # Llama/Qwen-style
print("ATTN:", blk.self_attn)         # has q_proj, k_proj, v_proj, o_proj
print("MLP:", blk.mlp)
target_modules = LORA_TARGET_MODULES
# LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]
# LORA_TARGET_MODULES = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

ATTN: Qwen3Attention(
  (q_proj): Linear4bit(in_features=2560, out_features=4096, bias=False)
  (k_proj): Linear4bit(in_features=2560, out_features=1024, bias=False)
  (v_proj): Linear4bit(in_features=2560, out_features=1024, bias=False)
  (o_proj): Linear4bit(in_features=4096, out_features=2560, bias=False)
  (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
  (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
)
MLP: Qwen3MLP(
  (gate_proj): Linear4bit(in_features=2560, out_features=9728, bias=False)
  (up_proj): Linear4bit(in_features=2560, out_features=9728, bias=False)
  (down_proj): Linear4bit(in_features=9728, out_features=2560, bias=False)
  (act_fn): SiLUActivation()
)


In [ ]:
lora_cfg = LoraConfig(
    task_type="CAUSAL_LM",
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=LORA_TARGET_MODULES,
    bias="none",
)
# model = PeftModel.from_pretrained(base, "HUGGING FACE ADAPTERS")
model = get_peft_model(base, lora_cfg)
model.print_trainable_parameters()

trainable params: 16,515,072 || all params: 4,038,983,168 || trainable%: 0.4089


In [ ]:

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    warmup_ratio=0.03,
    logging_steps=LOG_STEPS,
    save_steps=SAVE_STEPS,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_total_limit=2,
    bf16=torch.cuda.is_available(),
    optim="paged_adamw_8bit",
    report_to=["wandb"],
    load_best_model_at_end=True,
    hub_model_id=OUTPUT_DIR,
    hub_strategy="every_save",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.0
        )
    ],
)
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": True})
model.config.use_cache = False
model.config.attn_implementation = "flash_attention_2"

/tmp/ipython-input-1854523810.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print("Training...")
try:
  trainer.train(resume_from_checkpoint=False)
  eval_results = trainer.evaluate()
except Exception as e:
  print(f"Training failed: {e}")
finally:
  trainer.save_state()
  trainer.save_model(OUTPUT_DIR)
  print("Saved model to:", OUTPUT_DIR)

wandb.finish()
eval_loss = eval_results["eval_loss"]
perplexity = math.exp(eval_loss)
print(f"Eval loss = {eval_loss:.2f}, Perplexity = {perplexity:.2f}")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Training...


You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Casting fp32 inputs back to torch.bfloat16 for flash-attn compatibility.


Step,Training Loss,Validation Loss
30,1.315900,1.208461
60,1.026300,1.088340
90,0.885000,1.027849
120,0.811800,0.993668
150,0.895100,0.967579
180,0.751400,0.967662
210,0.676000,0.949786
240,0.763300,0.937692
270,0.637500,0.949643
300,0.551500,0.949635


Step,Training Loss,Validation Loss
30,1.315900,1.208461
60,1.026300,1.088340
90,0.885000,1.027849
120,0.811800,0.993668
150,0.895100,0.967579
180,0.751400,0.967662
210,0.676000,0.949786
240,0.763300,0.937692
270,0.637500,0.949643
300,0.551500,0.949635


...

  ...adapter_model.safetensors:  63%|######3   | 41.8MB / 66.1MB            

  ...T-Qwen3-4B/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

Saved model to: j05hr3d/SFT-Qwen3-4B


eval/loss,█▅▃▃▂▂▁▁▁▁▁▃▁▁▁
eval/runtime,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/samples_per_second,▁██████████████
eval/steps_per_second,▁██████████████
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/grad_norm,▁▂▄▃▂▅▃▄▃▆█▇▆▅
train/learning_rate,█▇▇▆▆▅▅▄▄▃▃▂▂▁
train/loss,█▅▄▄▄▃▂▃▂▁▂▁▁▁
eval/loss,0.9311
eval/runtime,31.7736


Eval loss = 0.93, Perplexity = 2.54


In [ ]:

# Quick smoke test generation (greedy/short)
model.eval()
prompt_msgs = [
    {"role": "system", "content": "Return exactly ONE fenced code block labeled `cryptol` and nothing else (no prose before/after)."},
    {"role": "user", "content": "Implement a function named `xor8` which takes two 8-bit words and returns their bitwise XOR. Also provide a constant `zero8` equal to 0 (8-bit)."}
]

# add_generation_prompt=True -> model should produce assistant continuation
inputs = tokenizer.apply_chat_template(
    prompt_msgs, tokenize=True, add_generation_prompt=True, return_tensors="pt"
).to(model.device)

with torch.no_grad():
    out = model.generate(
        inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )

generated = out[0, inputs.shape[-1]:]
print(tokenizer.decode(generated, skip_special_tokens=True))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Casting fp32 inputs back to torch.float16 for flash-attn compatibility.


```cryptol
xor8 : [8] -> [8] -> [8]
xor8 x y = x ^ y

zero8 : [8]
zero8 = 0
```


In [ ]:
runtime.unassign()


## Notes & References
- 🤗 Transformers `Trainer` docs (training loop, arguments)  
  https://huggingface.co/docs/transformers/main_classes/trainer
- `apply_chat_template` for model-specific chat formatting  
  https://huggingface.co/docs/transformers/main/chat_templating
- PEFT / LoRA docs  
  https://huggingface.co/docs/peft
- BitsAndBytes 4-bit quantization  
  https://github.com/TimDettmers/bitsandbytes
